<a href="https://colab.research.google.com/github/Chiraagkv/ASL/blob/main/holdmybeer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import random as rd
from PIL import Image
import PIL

In [2]:
!mkdir /root/.kaggle/
!touch /root/.kaggle/kaggle.json
!echo '{"username":"kingrohitkumar","key":"734c20a118d1339e94b4192d5fe26313"}' > /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d grassknoted/asl-alphabet --unzip

 99% 1.01G/1.03G [00:05<00:00, 172MB/s]
100% 1.03G/1.03G [00:05<00:00, 205MB/s]


In [4]:
base_model=hub.KerasLayer(handle='https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1', trainable=False, weights=None)
data=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,
                                                     samplewise_std_normalization=True,
                                                     height_shift_range=0.2,
                                                     width_shift_range=0.2,              
                                                     rotation_range=0.2,
                                                     zca_epsilon=1e-6,
                                                     shear_range=0.2,
                                                     zoom_range=0.2,
                                                     horizontal_flip=True,
                                                     cval=0.0)
data=data.flow_from_directory(directory='/content/sampled_image_path_main',
                              target_size=(224, 224),
                              class_mode='categorical',
                              batch_size=19,
                              shuffle=False,
                              interpolation='bilinear')


augmentation=tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.RandomCrop(2, 2),
                                  tf.keras.layers.experimental.preprocessing.RandomContrast(0.2)
])

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:356: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 8700 images belonging to 29 classes.


In [3]:
def randoms_images (image_path, duplicate_path):
  for (roots, dirs, filenames) in os.walk(image_path):
    for i in range (len(dirs)):
      os.makedirs(duplicate_path+'/'+dirs[i])
      cwd=image_path+'/'+dirs[i]
      for (root, dir, files) in os.walk(cwd):
        for m in range (300):
          image_name=files[m]
          img_path=Image.open(cwd+'/'+image_name)
          du=image_name.split('.')
          dup_path=duplicate_path+'/'+dirs[i]+'/'+f"{du[0]}.jpeg"
          img_path.save(dup_path)
          continue
        continue
      continue
randoms_images ('/content/asl_alphabet_train/asl_alphabet_train', 'sampled_image_path_main')

In [ ]:
inputs=tf.keras.layers.Input(shape=(224, 224, 3))

layer_1=augmentation(inputs)

model=base_model(inputs, training=True)

x2=tf.keras.layers.GlobalAveragePooling2D()(tf.expand_dims(tf.expand_dims(model, axis=0), axis=0))

outputs=tf.keras.layers.Dense(len(os.listdir('asl_alphabet_train')), activation='softmax')(x2)

model=tf.keras.Model(inputs, outputs)
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics='accuracy')
model.fit(data, epochs=1)